![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FTips&file=Python+Job+Parameters.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Tips/Python%20Job%20Parameters.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FTips%2FPython%2520Job%2520Parameters.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Tips/Python%20Job%20Parameters.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Tips/Python%20Job%20Parameters.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Python Job Parameters

Run your Python job `python script.py` ...

But, it needs inputs, or as we say parameters ...

How do we feed those parameters?

Two main directions
1. on the command line
2. import within the job
3. BONUS: both!
    
This notebook gives examples of getting parameters into Python Jobs.

---
## Setup

inputs:

In [1]:
EXPERIMENT = 'job-parms'
SERIES = 'tips'

parameters:

In [2]:
DIR = f'temp/{EXPERIMENT}'

environment:

In [3]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Example Job

In [4]:
%%writefile ./{DIR}/example.py

operation = 'add'
a = 2
b = 2.5
c = 10

def add(a, b, c):
    print(f'The value is: {a + b + c}')
    return a + b + c

def multiply(a, b, c):
    print(f'The value is: {a * b * c}')
    return a * b * c

if operation == 'add': add(a, b, c)
elif operation == 'multiply': multiply(a, b, c)

Writing ./temp/job-parms/example.py


In [5]:
!python ./{DIR}/example.py

The value is: 14.5


---
## Command Line Parameters

These come in several flavors:
- positional
    - optional
    - required
- flags
    - optional
    - required

**Goal For Examples Below:**

Modify the Example Job above to accept the four parameters from the command line:
- operation - make positional and optional, (default = add)
- a - make positional and required, int
- b - make a flag that is requires, float
- c - make a flag that is optional, float (default = 10) 

### Library: Argparse

- Use: `import argparse`
- [Reference](https://docs.python.org/3/library/argparse.html)

In [6]:
%%writefile ./{DIR}/example.py
import argparse

parser = argparse.ArgumentParser()
parser.add_argument(dest = 'operation', nargs = '?', default = 'add') # optional positional
parser.add_argument(dest = 'a', type = int) # required positional
parser.add_argument('--b', dest = 'b', type = float) # required flag
parser.add_argument('--c', dest = 'c', type = float, default = 10) # optional flag
args = parser.parse_args()

operation = args.operation
a = args.a
b = args.b
c = args.c

def add(a, b, c):
    print(f'The value is: {a + b + c}')
    return a + b + c

def multiply(a, b, c):
    print(f'The value is: {a * b * c}')
    return a * b * c

if operation == 'add': add(a, b, c)
elif operation == 'multiply': multiply(a, b, c)

Overwriting ./temp/job-parms/example.py


In [7]:
!python ./{DIR}/example.py 2 --b=2.5

The value is: 14.5


In [8]:
!python ./{DIR}/example.py --b=2.5 2

The value is: 14.5


In [9]:
!python ./{DIR}/example.py add 2 --b=2.5

The value is: 14.5


In [10]:
!python ./{DIR}/example.py --b=2.5 add 2

The value is: 14.5


In [11]:
!python ./{DIR}/example.py multiply 2 --b=2.5

The value is: 50.0


In [12]:
!python ./{DIR}/example.py multiply 2 --b=2.5 --c=1

The value is: 5.0


**Notes:**

- The positional parameters should all go before, or after, any flag parameters.  Just don't split them between before and after.

### Library: Docopt

- Use: `import docopt`
- [Reference](https://github.com/docopt/docopt)
- [docopt.org](http://docopt.org/)

Notes:
 - uses top of file to specify the usage of command line parameters
 - `[]` is an optional parameter
 - `()` is a required parameter
 - name that are upper-case or start+end with `<name>` are positional parameters
 - options can be used to desribe parameters
     - a description can follow the option but needs two spaces to deliniate
     - the desription can contain a default value, helpful for optional paramters, in the format [default: value-here]

In [13]:
%%writefile ./{DIR}/example.py
"""
usage:
    example.py OPERATION (A) (--b=B) [--c=C]
    example.py (A) (--b=B) [--c=C]

options:
    --b=B
    --c=C  Has value [default: 10]
"""
from docopt import docopt

def main(args):
    
    if args['OPERATION']:
        operation = args['OPERATION']
    else: operation = 'add'
    a = int(args['A'])
    b = float(args['--b'])
    c = float(args['--c'])
    
    def add(a, b, c):
        print(f'The value is: {a + b + c}')
        return a + b + c

    def multiply(a, b, c):
        print(f'The value is: {a * b * c}')
        return a * b * c

    if operation == 'add': add(a, b, c)
    elif operation == 'multiply': multiply(a, b, c)
    
if __name__ == '__main__':
    args = docopt(__doc__)
    #print(args)
    main(args)   

Overwriting ./temp/job-parms/example.py


In [14]:
!python ./{DIR}/example.py 2 --b=2.5

The value is: 14.5


In [15]:
!python ./{DIR}/example.py --b=2.5 2

The value is: 14.5


In [16]:
!python ./{DIR}/example.py add 2 --b=2.5

The value is: 14.5


In [17]:
!python ./{DIR}/example.py --b=2.5 add 2

The value is: 14.5


In [18]:
!python ./{DIR}/example.py multiply 2 --b=2.5

The value is: 50.0


In [19]:
!python ./{DIR}/example.py multiply 2 --b=2.5 --c=1

The value is: 5.0


### Library: Click

- Use: `import click`
- [Reference](https://palletsprojects.com/p/click/)

In [20]:
%%writefile ./{DIR}/example.py
import click

@click.command()
@click.option('--operation', default = 'add', type = click.STRING)
@click.argument('a', type = click.INT)
@click.option("--b", type = click.FLOAT)
@click.option("--c", default = 10, type = click.FLOAT)

def main(operation, a, b, c):
    
    def add(a, b, c):
        print(f'The value is: {a + b + c}')
        return a + b + c

    def multiply(a, b, c):
        print(f'The value is: {a * b * c}')
        return a * b * c

    if operation == 'add': add(a, b, c)
    elif operation == 'multiply': multiply(a, b, c)

if __name__ == '__main__':
    main()

Overwriting ./temp/job-parms/example.py


In [21]:
!python ./{DIR}/example.py 2 --b=2.5

The value is: 14.5


In [22]:
!python ./{DIR}/example.py --b=2.5 2

The value is: 14.5


In [23]:
!python ./{DIR}/example.py --operation=add 2 --b=2.5

The value is: 14.5


In [24]:
!python ./{DIR}/example.py --b=2.5 --operation=add 2

The value is: 14.5


In [25]:
!python ./{DIR}/example.py --operation=multiply 2 --b=2.5

The value is: 50.0


In [26]:
!python ./{DIR}/example.py --operation=multiply 2 --b=2.5 --c=1

The value is: 5.0


---
## Parameter Import

In [27]:
parameters = {
    'operation': 'add',
    'a': 2,
    'b': 2.5,
    'c': 10 
}

In [28]:
parameters

{'operation': 'add', 'a': 2, 'b': 2.5, 'c': 10}

### JSON

output:

In [29]:
import json

with open(f'./{DIR}/example.json', 'w') as file:
    json.dump(parameters, file)

input:

In [30]:
import json

with open(f'./{DIR}/example.json', 'r') as file:
    data = json.load(file)

show file contents:

In [31]:
from IPython.display import Markdown

with open(f'./{DIR}/example.json', 'r') as file:
    raw_data = file.read()
Markdown(f"```json\n\n{raw_data}\n```")

```json

{"operation": "add", "a": 2, "b": 2.5, "c": 10}
```

create job:

In [32]:
%%writefile ./{DIR}/example.py
import json

parameters = json.load(open('example.json', 'r'))
operation = parameters['operation']
a = parameters['a']
b = parameters['b']
c = parameters['c']

def add(a, b, c):
    print(f'The value is: {a + b + c}')
    return a + b + c

def multiply(a, b, c):
    print(f'The value is: {a * b * c}')
    return a * b * c

if operation == 'add': add(a, b, c)
elif operation == 'multiply': multiply(a, b, c)

Overwriting ./temp/job-parms/example.py


execute the script in the folder:

In [33]:
!ls ./{DIR}

example.json  example.py


In [34]:
!cd ./{DIR} && python example.py

The value is: 14.5


### YAML

output:

In [35]:
import yaml

with open(f'./{DIR}/example.yml', 'w') as file:
    yaml.dump(parameters, file)

input:

In [36]:
import yaml

with open(f'./{DIR}/example.yml', 'r') as file:
    data = yaml.full_load(file)

show file contents:

In [37]:
from IPython.display import Markdown

with open(f'./{DIR}/example.yml', 'r') as file:
    raw_data = file.read()
Markdown(f"```json\n\n{raw_data}\n```")

```json

a: 2
b: 2.5
c: 10
operation: add

```

create job:

In [38]:
%%writefile ./{DIR}/example.py
import yaml

parameters = yaml.full_load(open('example.yml', 'r'))
operation = parameters['operation']
a = parameters['a']
b = parameters['b']
c = parameters['c']

def add(a, b, c):
    print(f'The value is: {a + b + c}')
    return a + b + c

def multiply(a, b, c):
    print(f'The value is: {a * b * c}')
    return a * b * c

if operation == 'add': add(a, b, c)
elif operation == 'multiply': multiply(a, b, c)

Overwriting ./temp/job-parms/example.py


execute the script in the folder:

In [39]:
!ls ./{DIR}

example.json  example.py  example.yml


In [40]:
!cd ./{DIR} && python example.py

The value is: 14.5


### Pickle

output:

In [41]:
import pickle

with open(f'./{DIR}/example.pickle', 'wb') as file:
    pickle.dump(parameters, file)

input:

In [42]:
import pickle

with open(f'./{DIR}/example.pickle', 'rb') as file:
    data = pickle.load(file)

create job:

In [43]:
%%writefile ./{DIR}/example.py
import pickle

parameters = pickle.load(open('example.pickle', 'rb'))
operation = parameters['operation']
a = parameters['a']
b = parameters['b']
c = parameters['c']

def add(a, b, c):
    print(f'The value is: {a + b + c}')
    return a + b + c

def multiply(a, b, c):
    print(f'The value is: {a * b * c}')
    return a * b * c

if operation == 'add': add(a, b, c)
elif operation == 'multiply': multiply(a, b, c)

Overwriting ./temp/job-parms/example.py


execute the script in the folder:

In [44]:
!ls ./{DIR}

example.json  example.pickle  example.py  example.yml


In [45]:
!cd ./{DIR} && python example.py

The value is: 14.5
